**Описание проекта**

Заказчиком проекта является сервис, который предоставляет возможность отслеживать результаты спортсменов по  фигурному катанию на различных соревнованиях. Это может помочь тренерам выстроить грамотный план тренировок и соревнований для будущих чемпионов.
Необходимо создать решение по автоматическому опеределению от какой спортивной школы выступает спортсмен, основываясь на названии введенном в ручном порядке. Одна и та же школа может быть записана по-разному.
Необходимо построить модель, которая сможет сверять введенное название с его эталонным написанием.

**План**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
- Подготовить обучающий набор данных на основе эталонного датасета
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе
- Проанализировать результат и предложить варианты улучшения


**Описание данных**

Данные представлены в двух таблицах:
- Первая с эталонными названиями школ и регионами
- Вторая с примерами пользовательского ввода названий школ

# Загрузка библиотек и данных

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install nlpaug

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
from sklearn.model_selection import train_test_split
import warnings
import re
import nlpaug.augmenter as naw
from datasets import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, InputExample, losses, SentencesDataset
from sentence_transformers.util import semantic_search

In [ ]:
reference = pd.read_csv('/content/Школы.csv')

In [ ]:
sample = pd.read_csv('/content/Примерное написание.csv')

In [ ]:
def data_overview(df):

    # Вывод первых нескольких строк файла
    print("Первые несколько строк файла:")
    print(df.head())
    print("\n")

    # Вывод общей информации о DataFrame
    print("Информация о DataFrame:")
    print(df.info())
    print("\n")

    # Проверка пропусков (абсолютные и относительные значения)
    print("Проверка пропусков (абсолютные значения):")
    print(df.isnull().sum())
    print("\n")

    print("Проверка пропусков (относительные значения):")
    print(df.isnull().mean())
    print("\n")

    # Распределение данных для качественных (числовых) столбцов
    print("Распределение данных для числовых столбцов:")
    print(df.describe())
    print("\n")

    # Уникальные значения для категориальных столбцов
    print("Уникальные значения для категориальных столбцов:")
    for column in df.select_dtypes(include=['object', 'category']).columns:
        print(f"Столбец '{column}':")
        print(df[column].value_counts())
        print("\n")

In [ ]:
data_overview(reference)

Первые несколько строк файла:
   school_id                  name                region
0          1              Авангард    Московская область
1          2              Авангард     Ямало-Ненецкий АО
2          3               Авиатор  Республика Татарстан
3          4                Аврора       Санкт-Петербург
4          5  Ice Dream / Айс Дрим       Санкт-Петербург


Информация о DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB
None


Проверка пропусков (абсолютные значения):
school_id    0
name         0
region       0
dtype: int64


Проверка пропусков (относительные значения):
school_id    0.0
name         0.0
region       0.0
dtype: float64


Распределение данных для числов

In [ ]:
data_overview(sample)

Первые несколько строк файла:
   school_id                                               name
0       1836                                       ООО "Триумф"
1       1836                                Москва, СК "Триумф"
2        610                             СШОР "Надежда Губернии
3        610  Саратовская область, ГБУСО "СШОР "Надежда Губе...
4        609                                     "СШ "Гвоздика"


Информация о DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB
None


Проверка пропусков (абсолютные значения):
school_id    0
name         0
dtype: int64


Проверка пропусков (относительные значения):
school_id    0.0
name         0.0
dtype: float64


Распределение данных для числовых столбцов:
         school

**Вывод:** Данных для построения модели очень мало. Таблица с эталонными написаниями содержит только 305 записей. Таблица с размеченными произвольными написаниями названий школ содержит лишь 895 записей. Явные дубли отсутствуют

Приняли решение оставить таблицу с образцами для финального тестирования модели, а для обучения и отладки модели использовать таблицу с эталонными названиями.

# Обработка таблицы с эталонными названиями школ

Создадим новую таблицу для анализа данных и создания новых признаков

In [ ]:
features = reference

Проверим регионы в таблице


In [ ]:
sorted(features['region'].unique())

['Алтайский край',
 'Архангельская область',
 'Астраханская область',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Волгоградская область',
 'Вологодская область',
 'Воронежская область',
 'Забайкальский край',
 'Ивановская область',
 'Иркутская область',
 'Калининградская область',
 'Калужская область',
 'Кемеровская область',
 'Кировская область',
 'Костромская бласть',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Курганская область',
 'Курская область',
 'Ленинградская область',
 'Липецкая область',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Набережные челны',
 'Нижегородская область',
 'Новгородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская область',
 'Орловская область',
 'Пензенская область',
 'Пермский край',
 'Приморский край',
 'Псковская область',
 'Республика Башкортостан',
 'Республика Карелия',
 'Республика Коми',
 'Республика Корелия',
 'Республика Крым',
 'Республика Марий Эл

При анализе списка регионов, нашли ошибки в написании некоторых названий, что привело к появлению неявных дублей. Например - Карелия/Корелия

Так же есть регионы, которые написаны несколькими способами - Республика Чувашия/Чувашская республика.

Так же имеются в списке регионов города

Приняли решение заменить названия на эталонные.

In [ ]:
replacement_dict = {
    'Чувашская республика' : 'Чувашская Республика',
    'Республика Чувашия' : 'Чувашская Республика',
    'Республика Корелия' : 'Республика Карелия',
    'Удмуртская республика' : 'Удмуртская Республика',
    'Саранск' : 'Республика Мордовия',
    'Северодвинск ' : 'Архангельская область',
    'Костромская бласть' : 'Костромская область',
    'Набережные челны' : 'Республика Татарстан',
    'Ямало-Ненецкий АО' : 'Ямало-Ненецкий автономный округ',
    'ХМАО-Югра' : 'Ханты-Мансийский автономный округ - Югра',
    'Республика Саха' : 'Республика Саха'
    }

In [ ]:
features['region'] = features['region'].replace(replacement_dict)

In [ ]:
sorted(features['region'].unique())

['Алтайский край',
 'Архангельская область',
 'Астраханская область',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Волгоградская область',
 'Вологодская область',
 'Воронежская область',
 'Забайкальский край',
 'Ивановская область',
 'Иркутская область',
 'Калининградская область',
 'Калужская область',
 'Кемеровская область',
 'Кировская область',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Курганская область',
 'Курская область',
 'Ленинградская область',
 'Липецкая область',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Нижегородская область',
 'Новгородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская область',
 'Орловская область',
 'Пензенская область',
 'Пермский край',
 'Приморский край',
 'Псковская область',
 'Республика Башкортостан',
 'Республика Карелия',
 'Республика Коми',
 'Республика Крым',
 'Республика Марий Эл',
 'Республика Мордовия',
 'Республика Саха',
 'Республика Татарст

In [ ]:
features['region'].nunique()

66

После очистки названий в списке осталось 66 регионов

Приняли решение удалить из названий кавычки и некоторые формы организации, такие как ООО, ГБУ, МБУ, ГАУ, РФ, ДО, ФАУ, которые не несут смысловой нагрузки, но усложняют анализ данных.

In [ ]:
# Функция для очистки текста от лишних символов
def clean_text(text):
    #список аббевиатур для удаления
    abbreviations = ['ГАУ', 'МБУ', 'ГБУ', 'РФ', 'ГУ', 'ГБОУ', 'ДО', 'ФАУ', 'ООО', 'ИП']
    #регулярка для их удаления
    pattern = r'\b(?:' + '|'.join(abbreviations) + r')\b'
    text = re.sub(pattern, '', text).strip()
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\«\»\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # возвращаем очищенные данные
    return text

In [ ]:
# создаем список для хранения очищенных данных
cleaned_text = []
# для каждого названия из столбца name
for name in features['name']:
    # очищаем данные
    name = clean_text(name)
    # добавляем очищенные данные в список cleaned_text
    cleaned_text.append(name)
# записываем очищенные данные в новую колонку 'cleaned_text'
features['name'] = cleaned_text

Приняли решение просмотреть список школ, внутри регионов, для того чтобы исключить возможность появления дополнительных дублей

In [ ]:
features[['name','region']].duplicated().sum()

6

In [ ]:
features[features[['name','region']].duplicated()]

,school_id,name,region
11,12,Академия ФКК,Республика Мордовия
39,40,Голубева,Костромская область
67,68,Керриган,Республика Карелия
186,187,СШ 6,Мурманская область
213,214,СШОР 4,Чувашская Республика
286,289,Пируэт,Санкт-Петербург


В таблице нашлись 6 полных дубликата записей, отличающихся только id номером. Приняли решение их удалить

In [ ]:
ind_to_remove = [11, 39, 67, 186, 213, 289]
features = features.drop(ind_to_remove)
features = features.reset_index(drop=True)

In [ ]:
features = features.groupby('region').apply(lambda x: x.sort_values('name')).reset_index(drop=True)
pd.set_option('display.max_rows', None)
features

,school_id,name,region
0,305,Прогресс,Алтайский край
1,22,Беломорец,Архангельская область
2,9,Звездочка,Архангельская область
3,66,Каскад,Архангельская область
4,159,Созвездие,Астраханская область
5,51,ДЮСШ по ЗВС,Белгородская область
6,26,Брянск,Брянская область
7,36,СК,Брянская область
8,178,СШ по ФК,Брянская область
9,59,ДЮСШ 8,Владимирская область


Просмотрев весь список, отсортированный по регионам, мы нашли дополнительные дубли, не очевидные для машинного поиска. Приняли решение их удалить.

In [ ]:
ind_to_remove = [147, 148, 164, 255, 259, 136, 205, 200, 264, 271]
features = features.drop(ind_to_remove)
features = features.reset_index(drop=True)

In [ ]:
pd.reset_option('display.max_rows')

Теперь список очищен от дубликатов, которые можно обнаружить на первый взгляд и готов к дальнейшей работе.

# Создание новых признаков

Добавим в таблицу со школами сокращенные названия регионов - это данные взятые из "Правила вида спорта "стрельба из арбалета" (утв. приказом Минспорта России от 22.02.2019 N 157) раздел - СУБЪЕКТЫ РОССИЙСКОЙ ФЕДЕРАЦИИ, СОКРАЩЕННЫЕ НАЗВАНИЯ, ПРИНЯТЫЕ ДЛЯ ПРОТОКОЛОВ СОРЕВНОВАНИЙ.

Создам словарь для названия региона и его сокращения

In [ ]:
abbrev = pd.read_excel('/content/сокращения регионов.xlsx')

In [ ]:
abbrev.loc[79]

N                                        80
Субъект РФ                город Севастополь
Код для протокола                       СВС
Административный центр          Севастополь
Name: 79, dtype: object

In [ ]:
abbrev.head(5)

,N,Субъект РФ,Код для протокола,Административный центр
0,1,Республика Адыгея,АДГ,Майкоп
1,2,Республика Алтай,АЛТ,Горно-Алтайск
2,3,Республика Башкортостан,БАШ,Уфа
3,4,Республика Бурятия,БУР,Улан-Удэ
4,5,Республика Дагестан,ДАГ,Махачкала


Удалю из названия регионов слово город

In [ ]:
abbrev['Субъект РФ'] = abbrev['Субъект РФ'].str.replace('город ', '')

In [ ]:
abbrev['Субъект РФ'] = abbrev['Субъект РФ'].str.replace(r'\(.*?\)', '', regex=True)

In [ ]:
region_to_abbrev = dict(zip(abbrev['Субъект РФ'],abbrev['Код для протокола']))

In [ ]:
features['reg_abbrev'] = features['region'].map(region_to_abbrev)

In [ ]:
features.sample(3)

,school_id,name,region,reg_abbrev
123,32,Владлед,Приморский край,ПРК
192,275,ЛЕДОВАЯ ИСТОРИЯ,Санкт-Петербург,СПБ
260,86,Ледовый дворец,Тульская область,ТУЛ


In [ ]:
features[features['reg_abbrev'].isna()]

,school_id,name,region,reg_abbrev
146,140,РОО СФФК РСЯ,Республика Саха,NaN
147,142,РССШ по ЗВС,Республика Саха,NaN


In [ ]:
features['reg_abbrev'] = features['reg_abbrev'].fillna('САХ')
features[features['reg_abbrev'].isna()]

,school_id,name,region,reg_abbrev


Кроме таких признаных сокращений регионов часто используются просто первые буквы названий региона, создадим колонку с таким признаком. Республика Саха = РС

In [ ]:
def extract_first_letters(text):
  words = text.split() #разбиваем название региона на слова
  first_letters = [word[0].upper() for word in words]
  return ''.join(first_letters)

In [ ]:
features['first_letters'] = features['region'].apply(extract_first_letters)

In [ ]:
features.sample(3)

,school_id,name,region,reg_abbrev,first_letters
4,159,Созвездие,Астраханская область,АСТ,АО
191,79,Красная звезда,Санкт-Петербург,СПБ,С
233,81,Кристалл,Сахалинская область,САХ,СО


In [ ]:
features[features['first_letters'].str.len() == 1].sample(3)

,school_id,name,region,reg_abbrev,first_letters
224,254,ЦФКСиЗ Василеостровского района,Санкт-Петербург,СПБ,С
230,24,Школа ФК Е Бережной,Санкт-Петербург,СПБ,С
181,52,ДЮСШ СК Юбилейный,Санкт-Петербург,СПБ,С


У городов федерального значения из первых букв остается только одна - заменим это сокращение на тоже общепринятые.Для Москвы часто упротребимое сокращение Мос, для Санкт-Петербурга - СПб

In [ ]:
city_codes = {
    'Москва' : 'Мос',
    'Санкт-Петербург' : 'СПб'
}

def assign_code(city, code): #функция для замены значений для Москвы и Санкт-Петербурга
  if city in city_codes:
    return city_codes[city]
  else:
    return code #оставляем исходное значение, если город другой

features['first_letters'] = features.apply(
    lambda row: assign_code(row['region'], row['first_letters']), axis=1
    )

In [ ]:
features.sample(3)

,school_id,name,region,reg_abbrev,first_letters
71,211,СШОР 2,Москва,МСК,Мос
29,71,КО СШ по ЗВС,Калининградская область,КЛГ,КО
68,119,Олимп,Москва,МСК,Мос


Для дальнейшего сравнения создадим эталонное название составленное из Наименования школы и региона - title

Так же создадим новые признаки из сочетаний названия школы + региона в разном порядке и с сокращениями


In [ ]:
# Функция для создания новых столбцов по шаблону
def create_new_column(df, new_col_name, template):
    df[new_col_name] = df.apply(lambda row: template.format(**row), axis=1)

# Шаблоны для новых столбцов
templates = {
    'title': '{name} {region}',
    'name2': '{name} {reg_abbrev}',
    'name3': '{name} {first_letters}',
    'name4': '{region} {name}',
    'name5': '{reg_abbrev} {name}',
    'name6': '{first_letters} {name}',
}

# Создание новых столбцов на основе шаблонов
for new_col_name, template in templates.items():
    create_new_column(features, new_col_name, template)

In [ ]:
features.sample(10)

,school_id,name,region,reg_abbrev,first_letters,title,name2,name3,name4,name5,name6
112,121,Олимп,Оренбургская область,ОРЕ,ОО,Олимп Оренбургская область,Олимп ОРЕ,Олимп ОО,Оренбургская область Олимп,ОРЕ Олимп,ОО Олимп
101,103,Мещерский,Нижегородская область,НЖГ,НО,Мещерский Нижегородская область,Мещерский НЖГ,Мещерский НО,Нижегородская область Мещерский,НЖГ Мещерский,НО Мещерский
160,141,РОФФКК,Ростовская область,РСТ,РО,РОФФКК Ростовская область,РОФФКК РСТ,РОФФКК РО,Ростовская область РОФФКК,РСТ РОФФКК,РО РОФФКК
154,201,СШОР по ФККиШТ,Республика Татарстан,ТАТ,РТ,СШОР по ФККиШТ Республика Татарстан,СШОР по ФККиШТ ТАТ,СШОР по ФККиШТ РТ,Республика Татарстан СШОР по ФККиШТ,ТАТ СШОР по ФККиШТ,РТ СШОР по ФККиШТ
217,170,Стартайс,Санкт-Петербург,СПБ,СПб,Стартайс Санкт-Петербург,Стартайс СПБ,Стартайс СПб,Санкт-Петербург Стартайс,СПБ Стартайс,СПб Стартайс
98,186,СШ 6,Мурманская область,МУР,МО,СШ 6 Мурманская область,СШ 6 МУР,СШ 6 МО,Мурманская область СШ 6,МУР СШ 6,МО СШ 6
209,220,СШОР 1,Санкт-Петербург,СПБ,СПб,СШОР 1 Санкт-Петербург,СШОР 1 СПБ,СШОР 1 СПб,Санкт-Петербург СШОР 1,СПБ СШОР 1,СПб СШОР 1
192,275,ЛЕДОВАЯ ИСТОРИЯ,Санкт-Петербург,СПБ,СПб,ЛЕДОВАЯ ИСТОРИЯ Санкт-Петербург,ЛЕДОВАЯ ИСТОРИЯ СПБ,ЛЕДОВАЯ ИСТОРИЯ СПб,Санкт-Петербург ЛЕДОВАЯ ИСТОРИЯ,СПБ ЛЕДОВАЯ ИСТОРИЯ,СПб ЛЕДОВАЯ ИСТОРИЯ
4,159,Созвездие,Астраханская область,АСТ,АО,Созвездие Астраханская область,Созвездие АСТ,Созвездие АО,Астраханская область Созвездие,АСТ Созвездие,АО Созвездие
140,189,СШ 4,Республика Крым,КРМ,РК,СШ 4 Республика Крым,СШ 4 КРМ,СШ 4 РК,Республика Крым СШ 4,КРМ СШ 4,РК СШ 4


In [ ]:
features.shape

(290, 11)

In [ ]:
features.columns

Index(['school_id', 'name', 'region', 'reg_abbrev', 'first_letters', 'title',
       'name2', 'name3', 'name4', 'name5', 'name6'],
      dtype='object')

Добавлю так же признаки с ошибкой в написании названия школы

In [ ]:
# Функция для создания ошибок в строке
def introduce_typo(name):
    if len(name) > 1:
        pos = random.randint(0, len(name) - 1)
        random_char = random.choice('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ')
        name = name[:pos] + random_char + name[pos + 1:]
    return name


def create_columns_with_typos(df, columns_to_process, num_columns_with_typos=3):
    for col in columns_to_process:
        for i in range(1, num_columns_with_typos + 1):
            new_col_name = f'{col}with_typos{i}'
            df[new_col_name] = df[col].apply(lambda x: introduce_typo(x))

# Список колонок, для которых нужно создать ошибки
columns_to_process = ['title', 'name', 'name2', 'name3', 'name4', 'name5', 'name6']

# Создание новых колонок с ошибками
create_columns_with_typos(features, columns_to_process)

print(features.head(1))

   school_id      name          region reg_abbrev first_letters  \
0        305  Прогресс  Алтайский край        АЛТ            АК   

                     title         name2        name3  \
0  Прогресс Алтайский край  Прогресс АЛТ  Прогресс АК   

                     name4         name5  ... name3with_typos3  \
0  Алтайский край Прогресс  АЛТ Прогресс  ...      Нрогресс АК   

          name4with_typos1         name4with_typos2         name4with_typos3  \
0  Алтайский край Прозресс  Алтайский край ПрогреАс  Алшайский край Прогресс   

  name5with_typos1 name5with_typos2 name5with_typos3 name6with_typos1  \
0     АлТ Прогресс     АЛТ Нрогресс     АЛТ Прогрцсс      АК Пригресс   

  name6with_typos2 name6with_typos3  
0      Ао Прогресс      АК Прпгресс  

[1 rows x 32 columns]


In [ ]:
features.columns

Index(['school_id', 'name', 'region', 'reg_abbrev', 'first_letters', 'title',
       'name2', 'name3', 'name4', 'name5', 'name6', 'titlewith_typos1',
       'titlewith_typos2', 'titlewith_typos3', 'namewith_typos1',
       'namewith_typos2', 'namewith_typos3', 'name2with_typos1',
       'name2with_typos2', 'name2with_typos3', 'name3with_typos1',
       'name3with_typos2', 'name3with_typos3', 'name4with_typos1',
       'name4with_typos2', 'name4with_typos3', 'name5with_typos1',
       'name5with_typos2', 'name5with_typos3', 'name6with_typos1',
       'name6with_typos2', 'name6with_typos3'],
      dtype='object')

In [ ]:
features.shape

(290, 32)

In [ ]:
features.loc[features['school_id']==162]

,school_id,name,region,reg_abbrev,first_letters,title,name2,name3,name4,name5,...,name3with_typos3,name4with_typos1,name4with_typos2,name4with_typos3,name5with_typos1,name5with_typos2,name5with_typos3,name6with_typos1,name6with_typos2,name6with_typos3
141,162,Союз мастеров спорта,Республика Крым,КРМ,РК,Союз мастеров спорта Республика Крым,Союз мастеров спорта КРМ,Союз мастеров спорта РК,Республика Крым Союз мастеров спорта,КРМ Союз мастеров спорта,...,Союз мастеров Рпорта РК,Республика Крым СТюз мастеров спорта,Республика Крым Союзчмастеров спорта,РесЧублика Крым Союз мастеров спорта,КРМ Союз мастероб спорта,ГРМ Союз мастеров спорта,КРМ Союз мастеров мпорта,РК Союз мастпров спорта,РК Союз мастеров сцорта,РКяСоюз мастеров спорта


Создали достаточно данных, для того чтобы провести обучение модели. Удалим все строки, которые не подходят для обучения.

In [ ]:
features = features.drop(columns=['region', 'reg_abbrev', 'first_letters'])

In [ ]:
features.sample(3)

,school_id,name,title,name2,name3,name4,name5,name6,titlewith_typos1,titlewith_typos2,...,name3with_typos3,name4with_typos1,name4with_typos2,name4with_typos3,name5with_typos1,name5with_typos2,name5with_typos3,name6with_typos1,name6with_typos2,name6with_typos3
74,161,Сокольники,Сокольники Москва,Сокольники МСК,Сокольники Мос,Москва Сокольники,МСК Сокольники,Мос Сокольники,СокоФьники Москва,Сокольники МосквБ,...,Сокольникм Мос,Москяа Сокольники,Москва СокольникС,МоРква Сокольники,МСК СокольСики,МСК Сшкольники,МСК Сокольчики,Мос СокоРьники,Моь Сокольники,Мос Соколиники
247,175,СШ ЗВС,СШ ЗВС Ставропольский край,СШ ЗВС СТВ,СШ ЗВС СК,Ставропольский край СШ ЗВС,СТВ СШ ЗВС,СК СШ ЗВС,СШ ЗВС СтОвропольский край,СШ ЗВС Стаэропольский край,...,СШ ЗВС Сщ,Стайропольский край СШ ЗВС,Ставропольский край ТШ ЗВС,СтавропольскШй край СШ ЗВС,СТВ ЗШ ЗВС,тТВ СШ ЗВС,СТВжСШ ЗВС,СК СШ ЗНС,СО СШ ЗВС,Ср СШ ЗВС
274,95,МАУ СШ No 7,МАУ СШ No 7 Челябинская область,МАУ СШ No 7 ЧЕЛ,МАУ СШ No 7 ЧО,Челябинская область МАУ СШ No 7,ЧЕЛ МАУ СШ No 7,ЧО МАУ СШ No 7,МАУ СШ No 7 Челябёнская область,МАш СШ No 7 Челябинская область,...,МАУ СШ NoЗ7 ЧО,Чефябинская область МАУ СШ No 7,Челябинская омласть МАУ СШ No 7,Челябинская облъсть МАУ СШ No 7,ЧЕЛ МАУ СШ аo 7,ЧЕЛ МАУ СШ NС 7,ЧЕЛ МАУ СШ No 7,ЧО МАУ СШ вo 7,ЧО МАи СШ No 7,ЧО МАУ СШ фo 7


Преобразую широкую таблицу в вертикальную, сохранив верные идентификаторы школ.

In [ ]:
schools_features_melt = features.melt(id_vars=['school_id', 'title'], var_name='name_type', value_name='value')

In [ ]:
schools_features_melt = schools_features_melt.drop(columns=['name_type'])

In [ ]:
schools_features_melt = schools_features_melt.sample(frac=1).reset_index(drop=True)

In [ ]:
schools_features_melt.sample(5)

,school_id,title,value
3645,59,ДЮСШ 8 Владимирская область,ВО ДЮЕШ 8
3964,190,СШ 6 Республика Карелия,Республика Карелия СШ 6
5462,19,Арктур Ямало-Ненецкий автономный округ,Арктур Ямало-Ненецкий автономный окЧуг
6227,110,Наши надежды Московская область,Московская область Наши надежды
6142,247,Центр развития спорта Москва,Москва Центр развитиЕ спорта


In [ ]:
train, test = train_test_split(schools_features_melt, test_size=0.2, random_state=123)

In [ ]:
train.shape

(6264, 3)

In [ ]:
test.shape

(1566, 3)

**Вывод:** В процессе работы были созданы новые признаки для дальнейшей работы с машинным обучением. Из датасета, состоящего из 305 записей был создан обучающий набор из 7830 строк. Датасет был расширен путем добавления различных комбинаций из названия + региона, аббервиатуры региона, его сокращения, а так же путем внесения орфографических ошибок в получившееся наименование. Для дальнейшей работы приняли решение разделить его на обучающий и тестовый датасет в пропорции 80% на 20%.

# LaBSE from the box

Решила закодировать названия в эмбеддинги. Для кодирования использую модель LaBSE из sentence-transformers с huggingface. Эту модель можно использовать для отображения 109 языков в общее векторное пространство.
Создам модель и функцию для кодирования

In [ ]:
model = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

Кодируем целевые названия в эмбеддинги

In [ ]:
corpus = model.encode(features.title.values)

In [ ]:
query = model.encode(test.value.values)

In [ ]:
model_search = semantic_search(query, corpus, top_k=1)

In [ ]:
test['search_id'] = [x[0]['corpus_id'] for x in model_search]

In [ ]:
test['candidate_name'] = features.title.values[test.search_id.values]
test = test.drop(columns=['search_id'])

In [ ]:
test.sample(10)

,school_id,title,value,candidate_name
222,279,Созвездие Айс Санкт-Петербург,Санкт-Петейбург Созвездие Айс,Созвездие Айс Санкт-Петербург
2014,243,Холмск-Арена Сахалинская область,Сахалинская область Холмск-Арент,Холмск-Арена Сахалинская область
4153,219,СШОР 1 Республика Мордовия,Республика Мордовия СШОР 1,СШОР 1 Республика Мордовия
2706,119,Олимп Москва,МоФ Олимп,Олимп Москва
7730,78,Космос Калужская область,Космос КО,Космос Калужская область
2932,254,ЦФКСиЗ Василеостровского района Санкт-Петербург,Санкт-Петербург ЦФКСиЗ Василеъстровского района,ЦФКСиЗ Василеостровского района Санкт-Петербург
3739,104,МОСГОРСПОРТ Москва,МОСГОРСПОРТ,МОСГОРСПОРТ Москва
5028,151,Северная Олимпия Республика Коми,Северная ОлимпиЩ РК,Северная Олимпия Республика Коми
6549,192,СШОР Колпинского района Санкт-Петербург,СШОР Колпинского раТона СПБ,СШОР Колпинского района Санкт-Петербург
3848,136,ПроСинхро Москва,ПроСинЭро МСК,ПроСинхро Москва


In [ ]:
(test.title == test.candidate_name).sum()/test.shape[0]

0.8154533844189017

При использовании модели LaBSE без доработок, точность подбора правильного ответа составила 82%. Попробуем обучить модель работать именно с нашими данными.

# LaBSE training

In [ ]:
# Пример позитивных пар предложений
positive_pairs = train[['title', 'value']].values

# Создание списка InputExample
train_examples1 = [InputExample(texts=[s1, s2]) for s1, s2 in positive_pairs]

# Создание датасета и DataLoader
train_dataset = SentencesDataset(train_examples1, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# Определение функции потерь GISTEmbedLoss
guide_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
train_loss = losses.GISTEmbedLoss(model=model, guide=guide_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Использование модели SentenceTransformer для обучения
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

Step,Training Loss
500,1.735600


Проверка дообученноймодели на тестовых данных

In [ ]:
corpus = model.encode(features.title.values)

In [ ]:
query = model.encode(test.value.values)

In [ ]:
model_search = semantic_search(query, corpus, top_k=1)

In [ ]:
len(model_search)

1566

In [ ]:
test['search_id'] = [x[0]['corpus_id'] for x in model_search]

In [ ]:
test['candidate_name_train'] = features.title.values[test.search_id.values]
test = test.drop(columns=['search_id'])

In [ ]:
(test.title == test.candidate_name_train).sum()/test.shape[0]

0.9227330779054917

In [ ]:
test.sample(10)

,school_id,title,value,candidate_name,candidate_name_train
5558,265,Юность Ханты-Мансийский автономный округ - Югра,Ханты-Мансийский автономный офруг - Югра Юность,Юность Ханты-Мансийский автономный округ - Югра,Юность Ханты-Мансийский автономный округ - Югра
1405,1836,Триумф Москва,Триумф МСХ,Триумф Москва,Триумф Москва
2699,84,Ледовое поколение Орловская область,Оь Ледовое поколение,Ледовое поколение Орловская область,Ледовое поколение Орловская область
5588,30,Вилеса Айс Санкт-Петербург,Вилеса Айс,Вилеса Айс Санкт-Петербург,Вилеса Айс Санкт-Петербург
3180,283,Proice Kids Санкт-Петербург,Санкт-Пеоербург Proice Kids,Proice Kids Санкт-Петербург,Proice Kids Санкт-Петербург
5799,282,СШ ОРК филиала МО ЦСКА СКА г Санкт-Петербург С...,СПБ СШ ОРК филиала МО ЦСКА СКА г Санкт-Петербург,СШ ОРК филиала МО ЦСКА СКА г Санкт-Петербург С...,СШ ОРК филиала МО ЦСКА СКА г Санкт-Петербург С...
4548,46,Динамо Владимирская область,Владимирская облЪсть Динамо,Динамо Владимирская область,Динамо Владимирская область
6756,64,Золотые надежды Санкт-Петербург,ЗолотыеМнадежды СПб,Золотые надежды Санкт-Петербург,Золотые надежды Санкт-Петербург
2284,216,СШОР 6 Ростовская область,РО СШОР 6,СШОР 6 Ростовская область,СШОР 6 Ростовская область
4880,43,Движение Республика Татарстан,ТвТ Движение,Движение Республика Татарстан,Движение Республика Татарстан


In [ ]:
test[test['title'] != test['candidate_name_train']]

,school_id,title,value,candidate_name,candidate_name_train
809,134,Прибой Санкт-Петербург,Прибой,Прибой Санкт-Петербург,Прибой Тюменская область
2763,119,Олимп Москва,Олимп,Олимп Москва,Олимп Оренбургская область
2276,139,Рассвет Красноярский край,Красноярский крайЯРассвет,Рассвет Красноярский край,Красноярск Красноярский край
3307,158,Снеговик Краснодарский край,КК Енеговик,Конек Чайковской Москва,Конек Чайковской Москва
2868,175,СШ ЗВС Ставропольский край,СШ ЗВС Сщ,СШ ЗВС Красноярский край,СШ ЗВС Красноярский край
...,...,...,...,...,...
7743,173,СШ Ханты-Мансийский автономный округ - Югра,ХАО-Ю УШ,СШ Ханты-Мансийский автономный округ - Югра,Юность Ханты-Мансийский автономный округ - Югра
7241,126,ОЛИМПИЯ Свердловская область,ОЛИМПИЯ СО,ОЛИМПИЯ Санкт-Петербург,ОЛИМПИЯ Санкт-Петербург
1887,9,Звездочка Архангельская область,Звеадочка АО,АНО СК ЮНАРМЕЙЦЫ Севастополь,Юность Свердловская область
5855,203,СШОР 1 Оренбургская область,СШОИ 1,СШОР 1 Республика Коми,СШОР 1 Республика Коми


Дообученная модель показала отличный результат на тестовых данных, опледеляет верную школу с точностью 0.92. Некоторые аугументации исказили название школы до неузнаваемости, но так же проблемы в определении возникают, когда школы с аналогичными названиями присутствуют в разных регионах. Так например школа Ангелы Плющенко есть как в Москве, так и в Московской области.

Проверим модель на примере данных от заказчика

# LaBSE test on references

In [ ]:
school_sample = sample

In [ ]:
school_sample.sample(5)

,school_id,name
719,59,"Владимирская область, ДЮСШ №8"
785,38,"ЧОУ ДО ДЮСШ ""Голден Айс"""
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
867,14,"Московская область, МУ СШОР Альберта Демченко"
136,244,"ГБУ ДО МАФК, школа Хрустальный"


In [ ]:
school_sample.shape

(895, 2)

Для начала очищаем данные от символов в названиях

In [ ]:
# очищеннаем данные с помощью уже готовой функции
cleaned_text = []

for name in school_sample['name']:
    name = clean_text(name)
    cleaned_text.append(name)
school_sample['title'] = cleaned_text
school_sample = school_sample.drop(columns=['name'])

In [ ]:
school_sample.sample(5)

,school_id,title
115,248,Московская область Центр спортивных технологий
783,40,Костромская область КО СШОР им АВ Голубева
718,60,Иркутская обл МАУ АГО СШ Ермак
794,37,Нижегородская область НО СШОР по ЛВС
180,223,Санкт-Петербург КФК ТИТУЛ


Создадим новый корпус текстов для кодирования в эмбеддинги и дальнейшего семантического поиска, используя уже дообученную модель LaBSE

In [ ]:
query_sample = model.encode(school_sample.title.values)

In [ ]:
model_search_sample = semantic_search(query_sample, corpus, top_k=1)

In [ ]:
school_sample['search_id'] = [x[0]['corpus_id'] for x in model_search_sample]

In [ ]:
school_sample['score'] = [x[0]['score'] for x in model_search_sample]

In [ ]:
school_sample['candidate_name'] = features.title.values[school_sample.search_id.values]
school_sample['candidate_id'] = features.school_id.values[school_sample.search_id.values]
school_sample = school_sample.drop(columns=['search_id'])

In [ ]:
school_sample_merged = school_sample.merge(reference, left_on='school_id', right_on='school_id', how='left')

In [ ]:
school_sample_merged.sample(5)

,school_id,title,score,candidate_name,candidate_id,name,region
642,83,Ростовская область АНО СК ФКК Ледовая академия,0.796011,Ледовая академия Ростовская область,83,Ледовая академия,Ростовская область
250,203,Оренбургская область СШОР 1,0.967097,СШОР 1 Оренбургская область,203,СШОР 1,Оренбургская область
59,263,Мурманская область МАУ СШ Юность,0.827686,Юность Мурманская область,263,Юность,Мурманская область
626,88,Республика Тататрстан СШ Ледокол,0.789805,Ледокол Республика Татарстан,88,Ледокол,Республика Татарстан
767,46,Владимирская область ВРО ОГО ВФСО Динамо,0.756300,Динамо Владимирская область,46,Динамо,Владимирская область


In [ ]:
(school_sample.school_id == school_sample.candidate_id).sum()/school_sample.shape[0]

0.7195530726256983

На рабочем примере данных точность модели упала до 0.72, необходимо проанализировать в каких случаях модель допускает ошибки. Для большей наглядности можно присоединить датасет с эталонными названиями к таблице с образцами и вывести те строки в которых было дано ошибочное предсказание

In [ ]:
mistakes = school_sample_merged[school_sample_merged['school_id'] != school_sample_merged['candidate_id']]
mistakes = mistakes.sort_values(by='score')
mistakes.head(10)

,school_id,title,score,candidate_name,candidate_id,name,region
29,284,МО Коломяги Глайд,0.505180,Олимп Калужская область,118,ГЛАЙД,Санкт-Петербург
68,259,МО Финляндский АНО ФСО Чемпион,0.543609,АНО ШФК Республика Башкортостан,292,Чемпион,Санкт-Петербург
745,50,КОГАУ СШ Дымка,0.568605,ДЮСШ 2 Курганская область,55,Дымка,Кировская область
23,293,АНО Лига фигурного катания,0.578950,Армия фигурного катания Московская область,21,АНО Лига фигурного катания,Москва
853,19,ЯНАО МАУ СШ Арктур,0.579666,СШ Ханты-Мансийский автономный округ - Югра,173,Арктур,Ямало-Ненецкий автономный округ
32,279,МО Прометей КФК Созвездие Айс,0.584960,МосСпортОбъект ЛЦ Звезда Москва,294,Созвездие Айс,Санкт-Петербург
789,38,МО Коломяги ДЮСШ Голден айс,0.586051,МКУ ДЮСШ ВОСТОЧНАЯ Воронежская область,268,Голден айс,Санкт-Петербург
749,48,Санкт-Петрбург СПб ГБПОУ Академия ледовых видо...,0.600385,СШОР по фигурному катанию на коньках Санкт-Пет...,198,Динамо Санкт-Петербург,Санкт-Петербург
885,5,Айсдрим,0.606249,Айсберг Республика Крым,6,Ice Dream Айс Дрим,Санкт-Петербург
748,49,МО 7-й округ Династия,0.609474,ДЮСШ 7 Иркутская область,58,Династия,Санкт-Петербург


In [ ]:
mistakes.tail(10)

,school_id,title,score,candidate_name,candidate_id,name,region
830,26,Брянская область СК Брянск,0.928731,СК Брянская область,36,Брянск,Брянская область
832,26,Брянская область СК Брянск,0.928731,СК Брянская область,36,Брянск,Брянская область
828,26,Брянская область СК Брянск,0.928731,СК Брянская область,36,Брянск,Брянская область
288,197,РМ СШОР по ФКК Республика Мордовия,0.936705,СШОР по ФКК Республика Мордовия,200,СШОР по фигурному катанию на конька,Республика Мордовия
833,26,СК Брянск Брянская область,0.939026,СК Брянская область,36,Брянск,Брянская область
580,96,Москва Московская академия фигурного катания н...,0.952986,Московская академия фигурного катания на коньк...,107,МАФКК,Москва
582,96,Москва Московская академия фигурного катания н...,0.953139,Московская академия фигурного катания на коньк...,107,МАФКК,Москва
618,90,Свердловская обл МБОУ СШ 8 Локомотив,0.966430,МБОУ СШ 8 Локомотив Свердловская область,296,Локомотив,Свердловская область
298,194,Нижегородская область НО СШОР по ЛВС,0.973185,НО СШОР по ЛВС Нижегородская область,37,СШОР по ЛВС,Нижегородская область
616,90,Свердловская область МБОУ СШ 8 Локомотив,0.973302,МБОУ СШ 8 Локомотив Свердловская область,296,Локомотив,Свердловская область


После сортировки по уровню уверенности модели в полученном результате, можно сделать вывод, что в ситуациях, когда модель ошиблась, но показывала вероятность правильного ответа около 0.9 проблема видна либо в том, что названия очень близки, либо в эталонной таблице присутствуют скрытые дубликаты. Так например модель отностит "Свердловская область МБОУ СШ 8 Локомотив" к "МБОУ СШ 8 Локомотив Свердловская область", а по разметке подразумевалась школа Локомотив Свердловская область, что возможно является одной и той же школой. Так же например "Нижегородская область НО СШОР по ЛВС" отнесена к "НО СШОР по ЛВС Нижегородская область", а должна к СШОР по ЛВС Нижегородская область.

Из тех школ, которым модель присвоила id с трудом, некоторые школы из Санкт-Петербурга имеют районную приставку которая очень путает модель (СШ Фрунзенского района, Колпинского района, МО Финляндский, МО Коломяги), так как эти вариации названий отсутствовали в эталонном датасете.

# Анализ результата исследования

Было проведено исследование данных заказчика. Для анализа были предоставлены два небольших набора данных - с эталонными названиями школ и и с примерами их разного написания.

Так как данных для построения модели не достаточно было принято решение создать собственный датасет для обучения, используя эталонные наименования. Датасет был расширен путем добавления различных комбинаций из названия + региона, аббервиатуры региона, его сокращения, а так же путем внесения орфографических ошибок в получившееся наименование.

В результате таких комбинаций был подготовлен датасет на 7830 строк и разделен на тестовый и тренировочный.

В качестве модели были опробованы 2 модели DeepPavlov и LaBSE из sentence-transformers huggingface. DeepPavlov показала намного более долгий процесс обучения и более низкий результат в базовой версии. Поэтому была выбрана модель LaBSE. Эту модель можно использовать для отображения 109 языков в общее векторное пространство. Она очень хорошо работает так же с текстами на русском языке, но без доработки показала результат 82% точности.

Тренировочный датасет был использован для дообучения модели, после чего ее результат улучшился **до 92% точности**.

На данных заказчика метрика модели показала значение 72% точности.

После анализа ошибок, которые допустила модель на пользовательском вводе данных, можно сделать вывод, что в ситуациях, когда модель ошиблась, но показывала вероятность правильного ответа около 90% проблема видна либо в том, что названия очень близки, либо в эталонной таблице присутствуют скрытые дубликаты. Так например модель отностит "Свердловская область МБОУ СШ 8 Локомотив" к "МБОУ СШ 8 Локомотив Свердловская область", что возможно является одной и той же школой. Так же например "Нижегородская область НО СШОР по ЛВС" отнесена к "НО СШОР по ЛВС Нижегородская область", что возможно так же является одной и той же школой.

Из тех школ, которым модель присвоила id с трудом, некоторые школы из Санкт-Петербурга имеют районную приставку которая очень путает модель (СШ Фрунзенского района, Колпинского района, МО Финляндский, МО Коломяги), так как подобные варианты отсутствовали в эталонном датасете.

Это только некоторые примеры, в которых модель вероятно могла бы сделать верные предсказания, имея более выверенный датасет с эталонными названиями школ.

Так же проблемы в определении возникают, когда школы с аналогичными названиями присутствуют в разных регионах. Так например школа Ангелы Плющенко есть как в Москве, так и в Московской области.

**Модель, использованная в исследовании**
- SentenceTransformer('sentence-transformers/LaBSE').
- Так же для поиска реультата была использована утилита из sentence_transformers - semantic_search, которая сравнивает полученные в результате кодирования эмбеддинги и находит максимально похожую пару.

**Для оценки результата** использовалась метрика Точность (Accuracy) — она измеряет долю правильных предсказаний по отношению к общему числу предсказаний. Точность показывает, насколько хорошо модель или система распознаёт или предсказывает значения.

**Инструкция по запуску модели:**

In [ ]:
#очистить ваши данные от лишних символов и абберевиатур с помощью функции
#cleaned_text = []

#for name in ВАША_ТАБЛИЦА['name']:
    #name = clean_text(name)
    #cleaned_text.append(name)
#ВАША_ТАБЛИЦА['title'] = cleaned_text
#ВАША_ТАБЛИЦА = ВАША_ТАБЛИЦА.drop(columns=['name'])
#вместо колонки name появится колонка title с очищенным названием

In [ ]:
#query = model.encode(ВАША_ТАБЛИЦА.title.values)
#закодировать названия в эмбеддинги с помощью предобученной модели

In [ ]:
#model_search = semantic_search(query, corpus, top_k=1)
#провести поиск семантического сходства с эталонной таблицей, заранее записанной в corpus

In [ ]:
#ВАША_ТАБЛИЦА['search_id'] = [x[0]['corpus_id'] for x in model_search]
#ВАША_ТАБЛИЦА['candidate_name'] = features.title.values[ВАША_ТАБЛИЦА.search_id.values]
#ВАША_ТАБЛИЦА = ВАША_ТАБЛИЦА.drop(columns=['search_id'])
#добавляем в таблицу эталонное название, для наглядности

**Рекомендации заказчику**

Для дальнейшего улучшения работы модели необходимо доработать датасет с эталонными названиями, исключить дубли. Так же было бы полезно присвоить каждой школе какой-то уникальный код, например ИНН, который в дальнейшем могли бы указывать представители школ прирегистрации на соревнование. Это создаст задел на будущее, который упростит идентификацию бюджетных учреждений, так как именно они имеют самые сложные буквенные наименования.

In [ ]:
pip freeze > requirements.txt